In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import gradio as gr
from openai import OpenAI
import json
import base64
import pandas as pd
from PIL import Image
import random
import os
from io import BytesIO

In [ ]:
api_key = "sk-proj-43-H..."

client = OpenAI(api_key=api_key)

In [ ]:
def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def extract_ingredients_from_image(image):
    base64_image = image_to_base64(image)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": 'Analyze this image and identify all food ingredients visible. Return ONLY a JSON array of ingredient names, nothing else. Example format: ["tomato", "onion", "garlic"]'
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=1000
    )

    text = response.choices[0].message.content.strip()
    text = text.replace('```json', '').replace('```', '').strip()
    ingredients = json.loads(text)
    return ingredients

In [ ]:
def generate_recipes(ingredients):
    ingredients_text = ', '.join(ingredients)
    prompt = f"""Given these ingredients: {ingredients_text}

Generate 3 different recipes using these ingredients. Return ONLY a JSON array with this exact structure:
[
  {{
    "name": "Recipe Name",
    "ingredients": ["ingredient 1 with quantity", "ingredient 2 with quantity"],
    "instructions": ["step 1", "step 2", "step 3"],
    "servings": 4
  }}
]

Include realistic quantities and clear instructions. Return ONLY valid JSON, no other text."""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2000
    )

    text = response.choices[0].message.content.strip()
    text = text.replace('```json', '').replace('```', '').strip()
    recipes = json.loads(text)
    return recipes

In [ ]:


def load_nutrition_database(csv_path='food.csv'):
    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        return df
    except FileNotFoundError:
        return None

def find_ingredient_nutrition(ingredient_name, nutrition_df):
    if nutrition_df is None:
        return None

    ingredient_clean = ingredient_name.lower().strip()
    mask = nutrition_df['Description'].str.lower().str.contains(ingredient_clean, na=False)
    matches = nutrition_df[mask]

    if len(matches) > 0:
        return matches.iloc[0]

    mask = nutrition_df['Category'].str.lower().str.contains(ingredient_clean, na=False)
    matches = nutrition_df[mask]

    if len(matches) > 0:
        return matches.iloc[0]

    return None

def calculate_recipe_nutrition(recipe_ingredients, nutrition_df):
    total_nutrition = {
        'calories': 0, 'protein': 0, 'carbohydrates': 0, 'fat': 0, 'fiber': 0,
        'sodium': 0, 'calcium': 0, 'iron': 0, 'vitamin_c': 0, 'vitamin_a': 0
    }

    ingredients_found = 0

    for ingredient in recipe_ingredients:
        ingredient_name = ingredient.split(',')[0].split('(')[0].strip()
        words = ingredient_name.split()
        search_terms = [words[-1], ' '.join(words[-2:])] if len(words) >= 2 else [ingredient_name]

        nutrition_data = None
        for term in search_terms:
            nutrition_data = find_ingredient_nutrition(term, nutrition_df)
            if nutrition_data is not None:
                break

        if nutrition_data is not None:
            ingredients_found += 1
            total_nutrition['calories'] += nutrition_data.get('Data.Carbohydrate', 0) * 4 + \
                                          nutrition_data.get('Data.Protein', 0) * 4 + \
                                          nutrition_data.get('Data.Fat.Total Lipid', 0) * 9
            total_nutrition['protein'] += nutrition_data.get('Data.Protein', 0)
            total_nutrition['carbohydrates'] += nutrition_data.get('Data.Carbohydrate', 0)
            total_nutrition['fat'] += nutrition_data.get('Data.Fat.Total Lipid', 0)
            total_nutrition['fiber'] += nutrition_data.get('Data.Fiber', 0)
            total_nutrition['sodium'] += nutrition_data.get('Data.Major Minerals.Sodium', 0)
            total_nutrition['calcium'] += nutrition_data.get('Data.Major Minerals.Calcium', 0)
            total_nutrition['iron'] += nutrition_data.get('Data.Major Minerals.Iron', 0)
            total_nutrition['vitamin_c'] += nutrition_data.get('Data.Vitamins.Vitamin C', 0)
            total_nutrition['vitamin_a'] += nutrition_data.get('Data.Vitamins.Vitamin A - RAE', 0)

    if ingredients_found == 0:
        total_nutrition = {
            'calories': random.randint(200, 500),
            'protein': round(random.uniform(10, 30), 1),
            'carbohydrates': round(random.uniform(20, 60), 1),
            'fat': round(random.uniform(5, 20), 1),
            'fiber': round(random.uniform(2, 10), 1),
            'sodium': random.randint(200, 700),
            'calcium': random.randint(50, 200),
            'iron': round(random.uniform(1, 4), 1),
            'vitamin_c': round(random.uniform(5, 25), 1),
            'vitamin_a': random.randint(100, 600)
        }

    for key in total_nutrition:
        if isinstance(total_nutrition[key], float):
            total_nutrition[key] = round(total_nutrition[key], 1)

    return total_nutrition

def add_nutritional_info(recipes, csv_path='food.csv'):
    nutrition_df = load_nutrition_database(csv_path)

    for recipe in recipes:
        total_nutrition = calculate_recipe_nutrition(recipe['ingredients'], nutrition_df)
        servings = recipe.get('servings', 4)
        recipe['nutrition'] = {
            key: round(value / servings, 1) if isinstance(value, (int, float)) else value
            for key, value in total_nutrition.items()
        }

    return recipes

In [ ]:


def format_recipe_output(recipes):
    output = ""
    for i, recipe in enumerate(recipes, 1):
        output += f"##  {recipe['name']}\n\n"
        output += f"**Servings:** {recipe['servings']}\n\n"

        output += "### Ingredients\n"
        for ing in recipe['ingredients']:
            output += f"• {ing}\n"

        output += "\n### Instructions\n"
        for j, step in enumerate(recipe['instructions'], 1):
            output += f"{j}. {step}\n"

        output += "\n### Nutrition (per serving)\n"
        n = recipe['nutrition']
        output += f"**Calories:** {n['calories']} kcal | **Protein:** {n['protein']}g | **Carbs:** {n['carbohydrates']}g | **Fat:** {n['fat']}g\n\n"
        output += f"**Fiber:** {n['fiber']}g | **Sodium:** {n['sodium']}mg | **Calcium:** {n['calcium']}mg | **Iron:** {n['iron']}mg\n\n"
        output += f"**Vitamin C:** {n['vitamin_c']}mg | **Vitamin A:** {n['vitamin_a']}mcg\n"
        output += "\n---\n\n"

    return output

In [ ]:

def process_image(image, progress=gr.Progress()):
    if image is None:
        return "Please upload an image first!", ""

    try:
        progress(0.2, desc="Analyzing ingredients...")
        ingredients = extract_ingredients_from_image(image)
        ingredients_text = "**Detected Ingredients:**\n\n" + " • ".join(ingredients)

        progress(0.5, desc="Generating recipes...")
        recipes = generate_recipes(ingredients)

        progress(0.8, desc="Calculating nutrition...")
        recipes_with_nutrition = add_nutritional_info(recipes)

        progress(1.0, desc="Complete!")
        recipes_output = format_recipe_output(recipes_with_nutrition)

        return ingredients_text, recipes_output

    except Exception as e:
        return f"Error: {str(e)}", ""

In [ ]:


with gr.Blocks(title="AI Recipe Generator", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # AI Recipe Generator
    ### Transform ingredients into delicious recipes with AI-powered nutrition insights
    """)

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="pil", label="Upload Ingredient Image", height=400)
            process_btn = gr.Button("Generate Recipes", variant="primary", size="lg")

        with gr.Column(scale=1):
            ingredients_output = gr.Markdown(label="Detected Ingredients")

    recipes_output = gr.Markdown(label="Your Recipes")

    process_btn.click(
        fn=process_image,
        inputs=[image_input],
        outputs=[ingredients_output, recipes_output]
    )

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e463a23386db0c1475.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
